In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
company_name = "SOCIEDAD DE INVERSIONES DJ"
website = "https://registrosanitario.ispch.gob.cl/"

# Configure headless mode options and start browser
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # no gui environment
browser = webdriver.Chrome(options=options)

# Navigate to website
browser.get(website)

# checkbox id ctl00_ContentPlaceHolder1_chkTipoBusqueda_2
checkbox = browser.find_element(By.ID, "ctl00_ContentPlaceHolder1_chkTipoBusqueda_2")
checkbox.click()

# type company name in input id ctl00_ContentPlaceHolder1_txtEmpresa
company = browser.find_element(By.ID, "ctl00_ContentPlaceHolder1_txtEmpresa")
company.send_keys(company_name)

# search button: id ctl00_ContentPlaceHolder1_btnBuscar
search_button = browser.find_element(By.ID, "ctl00_ContentPlaceHolder1_btnBuscar")
search_button.click()

# Generate excel report: id ctl00_ContentPlaceHolder1_ImgBntExcel
excel_button = browser.find_element(By.ID, "ctl00_ContentPlaceHolder1_ImgBntExcel")
excel_button.click()

# Wait for Productos.xls to finish downloading
open("Productos.xls", "w").close()  # Productos.xls now exists but is empty
for i in range(40):  # 40 attempts over 20 seconds
    if os.path.getsize("Productos.xls") == 0:
        time.sleep(0.5)
    else:
        break # exit loop when file is not empty

# Close the browser window when done downloading
browser.quit()

In [ ]:
import pandas as pd

# Read the excel file
df = pd.read_html("Productos.xls")[0]

# drop the first column
df = df.drop(df.columns[0], axis=1)

# String to datetime
df["Fecha Registro"] = pd.to_datetime(df["Fecha Registro"])

In [ ]:
import requests
from urllib3.exceptions import InsecureRequestWarning

# Suppress only the single warning from urllib3 needed.
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

# Add a new column to store the upcoming renewal date
for registro in df["Registro"]:
    # Get the product details
    url = f"https://registrosanitario.ispch.gob.cl/Ficha.aspx?RegistroISP={registro}"
    response = requests.get(url, verify=False)
    
    # Find Fecha Próxima renovación by id=ctl00_ContentPlaceHolder1_lblProxRenovacion 
    upcoming_renewal_date = response.text.split('ctl00_ContentPlaceHolder1_lblProxRenovacion">')[1].split("</span>")[0]

    # save the upcoming_renewal_date in the dataframe
    df.loc[df["Registro"] == registro, "Fecha Próxima renovación"] = pd.to_datetime(upcoming_renewal_date, format="%d/%m/%Y")

    # set Fecha Vencimiento as Fecha Próxima renovación + 1 day. It must land on a business day
    df.loc[df["Registro"] == registro, "Fecha Vencimiento"] = pd.to_datetime(upcoming_renewal_date, format="%d/%m/%Y") + pd.offsets.CustomBusinessDay()

# Sort the dataframe by Fecha Próxima renovación
df = df.sort_values(by="Fecha Próxima renovación")

In [ ]:
# save the dataframe to a new csv, with timestamp
df.to_csv(f"{company_name} {time.strftime('%Y-%m-%d %H:%M:%S')}.csv", index=False)

In [ ]:
# clean everything
os.remove("Productos.xls")